# Sowing Success: How Machine Learning Helps Farmers Select the Best Crops

![Farmer in a field](farmer_in_a_field.jpg)

Measuring essential soil metrics such as nitrogen, phosphorous, potassium levels, and pH value is an important aspect of assessing soil condition. However, it can be an expensive and time-consuming process, which can cause farmers to prioritize which metrics to measure based on their budget constraints.

Farmers have various options when it comes to deciding which crop to plant each season. Their primary objective is to maximize the yield of their crops, taking into account different factors. One crucial factor that affects crop growth is the condition of the soil in the field, which can be assessed by measuring basic elements such as nitrogen and potassium levels. Each crop has an ideal soil condition that ensures optimal growth and maximum yield.

A farmer reached out to you as a machine learning expert for assistance in selecting the best crop for his field. They've provided you with a dataset called `soil_measures.csv`, which contains:

- `"N"`: Nitrogen content ratio in the soil
- `"P"`: Phosphorous content ratio in the soil
- `"K"`: Potassium content ratio in the soil
- `"pH"` value of the soil
- `"crop"`: categorical values that contain various crops (target variable).

Each row in this dataset represents various measures of the soil in a particular field. Based on these measurements, the crop specified in the `"crop"` column is the optimal choice for that field.  

In this project, you will build multi-class classification models to predict the type of `"crop"` and identify the single most importance feature for predictive performance.

In [46]:
# All required libraries are imported here for you.
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn import metrics

# Load the dataset
crops = pd.read_csv("soil_measures.csv")

# Write your code here

#view the data
crops.info();

#Choose metrics from confusion matrix

final_features =['N', 'P', 'K', 'ph']
X_train, X_test, y_train, y_test = train_test_split(
    crops[final_features],
    crops["crop"],
    test_size=0.2,
    random_state=42
)

log_reg = LogisticRegression(max_iter=2000,multi_class='multinomial')
log_reg.fit(X_train,y_train)
y_pred = log_reg.predict(X_test)

print(metrics.confusion_matrix(y_test, y_pred))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   N       2200 non-null   int64  
 1   P       2200 non-null   int64  
 2   K       2200 non-null   int64  
 3   ph      2200 non-null   float64
 4   crop    2200 non-null   object 
dtypes: float64(1), int64(3), object(1)
memory usage: 86.1+ KB
[[ 4  0  0  0  0  0  0 19  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 21  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 11  0  0  0  0  0  0  0  8  0  0  1  0  0  0  0  0  0  0  0]
 [ 0  0  0 26  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 18  0  0  0  0  0  0  0  8  0  0  0  0  0  0  1  0  0]
 [ 0  0  0  0  0 15  1  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 17  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 3  0  0  0  0  0  0 11  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1 

In [47]:
#Feature correlation

best_predictive_feature = ["", 0]

for f in ["N", "P", "K", "ph"]:
    model = LogisticRegression(max_iter=2000,multi_class='multinomial')
    model.fit(X_train[[f]],y_train)
    y_pred = model.predict(X_test[[f]])
    f1_score = metrics.f1_score(y_test,y_pred,average='macro')
    if len(best_predictive_feature) == 0 or best_predictive_feature[1] < f1_score: 
        best_predictive_feature[0] = f
        best_predictive_feature[1] = f1_score
    print(f"F1-score for {f}: {f1_score}")
    
best_predictive_feature = {best_predictive_feature[0] : best_predictive_feature[1]}
print(crops.corr())
print(best_predictive_feature)

F1-score for N: 0.10993898082576807
F1-score for P: 0.11919303408608646
F1-score for K: 0.208028683149612
F1-score for ph: 0.04671180311261425
           N         P         K        ph
N   1.000000 -0.231460 -0.140512  0.096683
P  -0.231460  1.000000  0.736232 -0.138019
K  -0.140512  0.736232  1.000000 -0.169503
ph  0.096683 -0.138019 -0.169503  1.000000
{'K': 0.208028683149612}


In [48]:
#Hyperparameter Tuning

# Define the model
model = LogisticRegression(max_iter=2000, multi_class='multinomial')

# Define the hyperparameters to tune
param_grid = {'C': np.linspace(0.1, 5, 20), 'solver': ['lbfgs']}

kf = KFold(n_splits=4, shuffle=True, random_state=42)

# Set up the grid search
grid_search = GridSearchCV(model, param_grid, cv=kf, scoring='f1_macro')

# Perform the grid search on your data
grid_search.fit(X_train, y_train)

# Print the best parameters
print("Best Parameters: ", grid_search.best_params_)
print("Best Result: ", grid_search.best_score_)



Best Parameters:  {'C': 3.710526315789474, 'solver': 'lbfgs'}
Best Result:  0.6803345004823224


In [49]:
#declare model

X = crops.drop("crop", axis=1)

log_reg = LogisticRegression(max_iter=2000, solver='lbfgs', C=3.710526315789474, multi_class='multinomial')

log_reg.fit(X_train,y_train)
y_pred = log_reg.predict(X_test)

#validate model
scores = cross_val_score(model, X, y, cv=kf, scoring='f1_macro')
print(scores)


[0.67620236 0.67992578 0.65516976 0.6445929 ]
